# Mix-in クラスで機能合成を考える

Python は多重継承を扱いやすくする組み込み機能を備えたオブジェクト指向言語
だが、多重継承は避けたほうが賢明

多重継承による簡便さとカプセル化が望ましいですが、面倒を避けたい場合、代わりに mix-in を書くことを考えた方がよい

In [8]:
class ToDictMixin:
  def to_dict(self):
    return self._traverse_dict(self.__dict__)
  
  def _traverse_dict(self, instance_dict):
    output = {}
    for key, value in instance_dict.items():
      output[key] = self._traverse(key, value)
    return output

  def _traverse(self, key, value):
    if isinstance(value, ToDictMixin):
      return value.to_dict()
    elif isinstance(value, dict):
      return self._traverse_dict(value)
    elif isinstance(value, list):
      return [self._traverse(key, i) for i in value]
    elif hasattr(value, '__dict__'):
      return self._traverse_dict(value.__dict__)
    else:
      return value

In [9]:
class BinaryTree(ToDictMixin):
  def __init__(self, value, left=None, right=None):
    self.value = value
    self.left = left
    self.right = right

In [10]:
tree = BinaryTree(10,
                  left=BinaryTree(7, right=BinaryTree(9)),
                  right=BinaryTree(13, left=BinaryTree(11)))
print(tree.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None}}, 'right': {'value': 13, 'left': {'value': 11, 'left': None, 'right': None}, 'right': None}}


mix-in の一番良いところは、ジェネリックな機能がプラグインとして扱えるようになり、
必要な時に振る舞いをオーバーライドできること

In [11]:
class BinaryTreeWithParent(BinaryTree):
  def __init__(self, value, left=None, right=None, parent=None):
    super().__init__(value, left, right)
    self.parent = parent

  def _traverse(self, key, value):
    if (isinstance(value, BinaryTreeWithParent) and
        key == 'parent'):
      return value.value # サイクルを防ぐ
    else:
      return super()._traverse(key, value)

In [12]:
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)
print(root.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None, 'parent': 7}, 'parent': 10}, 'right': None, 'parent': None}


In [13]:
class NamedSubTree(ToDictMixin):
  def __init__(self, name, tree_with_parent):
    self.name = name
    self.tree_withparent = tree_with_parent

my_tree = NamedSubTree('foobar', root.left.right)
print(my_tree.to_dict()) # 無限ループにならない

{'name': 'foobar', 'tree_withparent': {'value': 9, 'left': None, 'right': None, 'parent': 7}}


In [14]:
import json

class JsonMixin:
  @classmethod
  def from_json(cls, data):
    kwargs = json.loads(data)
    return cls(**kwargs)
  
  def to_json(self):
    return json.dumps(self.to_dict())

In [15]:
class DatacenterRack(ToDictMixin, JsonMixin):
  def __init__(self, switch=None, machines=None):
    self.switch = Switch(**switch)
    self.machines = [
      Machine(**kwargs) for kwargs in machines
    ]

class Switch(ToDictMixin, JsonMixin):
  def __init__(self, ports=None, speed=None):
    self.ports = ports
    self.speed = speed

class Machine(ToDictMixin, JsonMixin):
  def __init__(self, cores=None, ram=None, disk=None):
    self.cores = cores
    self.ram = ram
    self.disk = disk

In [18]:
serialised = """{
    "switch": {"ports": 5, "speed": 1e9},
    "machines": [
        {"cores": 8, "ram": 32e9, "disk": 5e12},
        {"cores": 4, "ram": 16e9, "disk": 1e12},
        {"cores": 2, "ram": 4e9, "disk": 500e9}
    ]
}"""

deserialized = DatacenterRack.from_json(serialised)
roundtrip = deserialized.to_json()
assert json.loads(serialised) == json.loads(roundtrip)